In [1]:
# import numpy as np
# df=np.load("/Users/rohitviswam/Downloads/CSI-02-0005/H_32T4R_30_1RB.npy")
# data=df.reshape(2100,398,256)
# data.shape
# feature_len=data.shape[-1]

In [1]:

import scipy.io
# Load the .mat file
file_path = '../EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']

# Check the shape and structure of the extracted data
data = EV_data
del EV_data
data.shape

(50, 1000, 832)

In [2]:
# data = data[:5]

In [3]:
data.shape

(50, 1000, 832)

In [4]:
import numpy as np
import torch
from iTransformer import iTransformer
from iTransformer import iTransformer2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
# data_flattened = data.reshape(data.shape[0], -1)  # shape becomes (2100, 398*256)


In [7]:
# data = data_flattened

In [8]:
import numpy as np
import torch

# Assuming `data` is already loaded and is of shape (2100, 398, 256)

def preprocess_UE_data(data, lookback_len=10):
    """
    Preprocesses data to create input-target pairs.
    Args:
        data: numpy array, (2100, 398, 256), original dataset
        lookback_len: int, number of past timesteps to consider as input
    
    Returns:
        inputs: torch Tensor, shape (samples, lookback_len, features)
        targets: torch Tensor, shape (samples, features)
    """
    num_UEs, num_timesteps, num_features = data.shape
    input_list = []
    target_list = []

    for ue in range(num_UEs):
        ue_data = data[ue]
        for i in range(num_timesteps - lookback_len):
            input_seq = ue_data[i: i + lookback_len]
            target_seq = ue_data[i + lookback_len]
            
            input_list.append(input_seq)
            target_list.append(target_seq)

    inputs = torch.tensor(input_list, dtype=torch.float32)
    targets = torch.tensor(target_list, dtype=torch.float32)

    return inputs, targets

# Usage example
inputs, targets = preprocess_UE_data(data, lookback_len=10)

print("Input shape:", inputs.shape)   # Expected: (samples, 10, 256)
print("Target shape:", targets.shape) # Expected: (samples, 256)


Input shape: torch.Size([4950, 10, 832])
Target shape: torch.Size([4950, 832])


/tmp/ipykernel_17607/93298084.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  inputs = torch.tensor(input_list, dtype=torch.float32)


In [9]:
import torch
import torch.nn as nn
from iTransformer import iTransformer

# Set up parameters and data
num_UEs, num_timesteps, num_features = data.shape
lookback_len = 10
forecast_len = 1


# Define the iTransformer model
model = iTransformer2D(
    num_variates=num_features,
    lookback_len=lookback_len,
    dim=832,           # Model dimension
    depth=6,           # Number of layers
    heads=8,           # Number of attention heads
    dim_head=64,       # Dimension per head
    pred_length=forecast_len,  # Prediction horizon (1 in this case)
    num_time_tokens=2,  # Single token per variate
    use_reversible_instance_norm=True
)



Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [10]:
from torchinfo import summary

# Assuming your model object is already defined as `model`

# Generate a model summary (batch size and input dimensions as required)
model_summary = summary(model, input_size=(4, 10, 832))  # Adjust batch size and input size accordingly

# This will print a detailed summary with parameters, layers, and dimensions
print(model_summary)


Layer (type:depth-idx)                             Output Shape              Param #
iTransformer2D                                     [4, 1, 832]               3,328
├─RevIN: 1-1                                       [4, 832, 10]              1,664
├─Sequential: 1-2                                  [4, 832, 6, 832]          --
│    └─Rearrange: 2-1                              [3328, 1, 10]             --
│    └─ConstantPad1d: 2-2                          [3328, 1, 15]             --
│    └─Conv1d: 2-3                                 [3328, 832, 6]            9,152
│    └─Rearrange: 2-4                              [4, 832, 6, 832]          --
│    └─LayerNorm: 2-5                              [4, 832, 6, 832]          1,664
├─Sequential: 1-3                                  [4, 832, 832]             --
│    └─Linear: 2-6                                 [4, 832, 832]             9,152
│    └─LayerNorm: 2-7                              [4, 832, 832]             1,664
├─ModuleList: 1-4

In [11]:
# Print the architecture by directly printing the model object
print(model)


iTransformer2D(
  (reversible_instance_norm): RevIN()
  (layers): ModuleList(
    (0-5): 6 x ModuleList(
      (0): SimpleGateLoopLayer(
        (norm): RMSNorm()
        (to_qkva): Sequential(
          (0): Linear(in_features=832, out_features=2496, bias=False)
          (1): Rearrange('b n (qkva d) -> qkva (b d) n 1', qkva=3)
        )
        (maybe_post_ln): Identity()
        (split_heads): Rearrange('(b d) n 1 -> b n d', d=832)
      )
      (1): TransformerBlock(
        (rotary_emb): RotaryEmbedding()
        (attn): Attention(
          (rotary_emb): RotaryEmbedding()
          (to_qkv): Sequential(
            (0): Linear(in_features=832, out_features=1536, bias=False)
            (1): Rearrange('b n (qkv h d) -> qkv b h n d', qkv=3, h=8)
          )
          (to_v_gates): Sequential(
            (0): Linear(in_features=832, out_features=512, bias=False)
            (1): SiLU()
            (2): Rearrange('b n (h d) -> b h n d', h=8)
          )
          (attend): Attend(
 

In [12]:
# Define training settings
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")

model.to(device)

inputs = inputs.to(device)
targets = targets.to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
batch_size = 32

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    num_batches = int(len(inputs) / batch_size)
    
    for batch in range(num_batches):
     batch_inputs = inputs[batch * batch_size:(batch + 1) * batch_size]
     batch_targets = targets[batch * batch_size:(batch + 1) * batch_size]

     optimizer.zero_grad()
     preds = model(batch_inputs)[forecast_len].squeeze(1)  # Remove the singleton dimension
     loss = criterion(preds, batch_targets)
     loss.backward()
     optimizer.step()
     epoch_loss += loss.item()


    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / num_batches:.4f}")


: 

In [ ]:
import pandas as pd

def tabulate_predictions(inputs, targets, model, forecast_len=1, batch_size=64):
    """
    Tabulates actual vs. predicted results.
    Args:
        inputs: torch Tensor, input data used to predict
        targets: torch Tensor, actual target values
        model: trained PyTorch model for making predictions
        forecast_len: int, how many timesteps to forecast
        batch_size: int, batch size to use in prediction

    Returns:
        DataFrame containing actual and predicted results.
    """
    model.eval()
    all_preds = []
    all_targets = []

    num_batches = int(len(inputs) / batch_size)

    with torch.no_grad():
        for batch in range(num_batches):
            batch_inputs = inputs[batch * batch_size:(batch + 1) * batch_size]
            batch_targets = targets[batch * batch_size:(batch + 1) * batch_size]

            # Predict using the model
            preds = model(batch_inputs)[forecast_len].squeeze(1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(batch_targets.cpu().numpy())

    # Create DataFrame for easier tabulation and analysis
    df_results = pd.DataFrame({
        'Actual': [list(target) for target in all_targets],
        'Predicted': [list(pred) for pred in all_preds]
    })

    return df_results

# Example Usage:
results_df = tabulate_predictions(inputs, targets, model, forecast_len=1, batch_size=64)
print(results_df.head())  # Print the first few rows to verify


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_predictions(inputs, targets, model, forecast_len=1, num_samples=10, batch_size=64):
    """
    Plots a comparison between actual and predicted values.
    Args:
        inputs: torch Tensor, input data used to predict
        targets: torch Tensor, actual target values
        model: trained PyTorch model for making predictions
        forecast_len: int, how many timesteps to forecast
        num_samples: int, how many samples to plot
        batch_size: int, batch size to use in prediction

    Returns:
        None. Displays the plots.
    """
    model.eval()
    predictions = []
    actuals = []

    num_batches = min(int(len(inputs) / batch_size), num_samples)

    with torch.no_grad():
        for batch in range(num_batches):
            batch_inputs = inputs[batch * batch_size:(batch + 1) * batch_size]
            batch_targets = targets[batch * batch_size:(batch + 1) * batch_size]

            preds = model(batch_inputs)[forecast_len].squeeze(1)
            predictions.extend(preds.cpu().numpy())
            actuals.extend(batch_targets.cpu().numpy())

    # Convert to numpy arrays for easy plotting
    predictions = np.array(predictions)
    actuals = np.array(actuals)

    # Plot each sample
    plt.figure(figsize=(15, 8))
    for i in range(num_samples):
        plt.subplot(num_samples // 2, 2, i + 1)
        plt.plot(actuals[i], label='Actual')
        plt.plot(predictions[i], label='Predicted')
        plt.legend()
        plt.xlabel('Feature Index')
        plt.ylabel('Value')
        plt.title(f'Sample {i + 1}')

    plt.tight_layout()
    plt.show()

# Example usage:
plot_predictions(inputs, targets, model, forecast_len=1, num_samples=2, batch_size=64)
